## El Protocolo de Contexto de Modelo (MCP) y la nueva era de los agentes inteligentes

A finales de **2024**, **Anthropic** lanzó el **Protocolo de Contexto de Modelo (MCP)** con gran éxito. Anthropic no solo estandarizó la forma de transformar los **chatbots** de simples hablantes a **agentes capaces de actuar**, sino que también lanzó MCP simultáneamente con el soporte de **IDEs populares basados en IA**, así como de **Claude Desktop** de Anthropic, lo que impulsó su rápida adopción.

Uno de los componentes principales de la arquitectura MCP, los **servidores MCP**, se diseñó para ser **increíblemente fácil de implementar**, pero **extremadamente complejo cuando se requerían flujos de trabajo avanzados**.  
Esto condujo a una rápida proliferación de servidores MCP creados por usuarios, lo que mejoró los flujos de trabajo de **IA generativa** y consolidó la adopción de MCP como el **estándar para la creación de agentes de IA**.

---

### ¿Qué es un agente?

Todo el mundo parece tener una opinión sobre qué es o no es un agente, pero es difícil hablar de agentes sin llegar a una **definición común**.  
Para los propósitos de este curso, usaremos la definición usada por Anthropic en su artículo *Building Effective Agents*:

> Un agente es un "sistema... donde grandes modelos de lenguaje (LLMs) dirigen dinámicamente sus propios procesos y el uso de herramientas, manteniendo el control sobre cómo realizan las tareas."

---

### Funcionamiento general de un agente

Generalmente, los agentes se implementan como **LLMs** que reciben una tarea y luego usan **herramientas** —cualquier código "externo" al agente que es llamado por éste para realizar alguna tarea— para **ejecutar una acción y recopilar retroalimentación** antes de continuar.

La recopilación de acciones y comentarios se produce en un **bucle iterativo**, donde:

1. El **LLM realiza una acción**.  
2. Se **recopilan comentarios** sobre esa acción.  
3. Los comentarios se **envían de vuelta al LLM**.  
4. El LLM decide si:
   - Presenta el resultado final al usuario.  
   - Utiliza los comentarios para realizar **otra acción**.  
   - O solicita **más retroalimentación** al usuario.

Este ciclo permite que el agente aprenda, corrija y mejore su rendimiento dinámicamente durante la ejecución de una tarea.


<p align="center">
<img src="../images/agent-feedback-loop.png" width="600" height="150">
    Figura 1-1. Un ciclo hipotético de acción-retroalimentación en acción para un agente de codificación.
</p>

## El rol del Model Context Protocol en la IA Agéntica

Antes de la introducción de **MCP**, la **IA agéntica** aún estaba en sus primeras etapas.  
Las organizaciones que construían plataformas internas de **IA generativa**, junto con frameworks públicos como **LangChain**, tenían que **inventar sus propios métodos para exponer herramientas a los LLM**.

El desafío era que **cada LLM requería su propio conector personalizado**.  
Esto creó lo que se conoce como el *“problema M×N”*: por cada uno de los **M** modelos de lenguaje que debían ser soportados, se necesitaban **N** conectores para las herramientas, fuentes de datos y otras integraciones.

Por ejemplo:

> Dar soporte a **3 modelos** y **4 herramientas** significaba tener que escribir **12 conectores separados**,  
> muchos de ellos repitiendo lógica similar,  
> lo que multiplicaba la complejidad y hacía que los sistemas fueran más propensos a **errores difíciles de diagnosticar**.


<img src="../images/prompt-chaining-example.png" width="600" height="150">
Figura 1-2. Un flujo de trabajo hipotético de encadenamiento de indicaciones para un agente de detección y traducción de idiomas.



<img src="../images/translation-example-agent.png" width="600" height="150">
Figura 1-3. Un agente de detección y traducción de idiomas.


**El Model Context Protocol, inspirado en el Language Server Protocol (LSP), proporciona una interfaz común para construir herramientas, prompts y recursos de datos, y conectarlos con los LLM.** De esta forma, para cualquier número **M** de LLM, solo es necesario escribir **N** conectores, transformando el problema de **M×N** en **M+N**. En nuestro ejemplo de 3 modelos y 4 herramientas, esto significaría que únicamente se necesitan 7 conectores en lugar de 12.



<img src="../images/mxn-problem.png" width="600" height="150">
Figura 1-4. Ilustración del problema MxN, donde 3 modelos necesitan acceso a 4 herramientas diferentes. Esto resulta en la necesidad de 12 conectores (4 x 3).

Figura 1-5. Con MCP proporcionando una interfaz común entre las herramientas y los modelos, ahora solo necesitamos 7 conectores para dar a los tres modelos acceso a las cuatro herramientas en lugar de 12. Cada modelo o aplicación de IA únicamente necesita su propio conector hacia el servidor MCP, y MCP ofrece una interfaz común para que todos ellos utilicen las herramientas disponibles.


Al crear una interfaz sencilla y estándar para que los modelos usen herramientas, los servidores MCP convierten el problema **M×N** en un problema **M+N**. Esto significa que cualquier modelo que pueda usar herramientas se conecta a un servidor MCP a través de un cliente MCP, y basta con un solo conector (el cliente) para acceder a todas las herramientas que ese servidor ofrece.

Cada servidor MCP que soporta herramientas expone dos funciones principales:

* **ListTools**: muestra qué herramientas están disponibles en el servidor.
* **CallTool**: permite llamar y usar una de esas herramientas.

La aplicación anfitriona, mediante su cliente MCP, utiliza estos dos puntos de acceso para descubrir las herramientas y ejecutarlas cuando lo necesite.



---

__Componentes MCP__

Los componentes principales de MCP son el servidor MCP, el cliente MCP y el transporte MCP. El servidor proporciona herramientas, indicaciones, recursos de datos y más a través de una interfaz común. El cliente es el conector que conecta una aplicación host con un servidor MCP. El transporte es el protocolo de comunicación subyacente que permite la comunicación entre el cliente y el servidor.

---



### El génesis del MCP: cómo nació un protocolo que transformó la integración de IA

Todo comenzó en Anthropic, donde David Soria Parra trabajaba como ingeniero de software usando Claude Desktop para desarrollar herramientas de programación asistida por IA. Como muchos que habían trabajado con programación asistida antes de la ola actual de LLMs, David se frustraba al tener que copiar y pegar constantemente código entre Claude Desktop y su editor de código. Esa molestia cotidiana lo llevó a preguntarse: ¿cómo podríamos simplificar esto de forma sistemática?

La inspiración llegó de su experiencia previa con el **Language Server Protocol (LSP)**, un estándar abierto que permite a los IDEs comunicarse con servidores de lenguaje para ofrecer funciones como autocompletado, definición de variables o renombrado de funciones. LSP resolvía problemas similares en el mundo de los IDEs, y David vio una analogía clara: así como LSP estandariza la comunicación entre IDEs y servidores de lenguaje, un protocolo similar podría estandarizar la interacción entre LLMs y herramientas.

El problema que quería resolver era el conocido **M×N**: para cada LLM que quisieras soportar, necesitabas N conectores para cada herramienta, creando un desorden de conectores repetidos y difícil de mantener. David comprendió que, con un protocolo adecuado, esto podía transformarse en un **problema M+N**, donde los conectores solo necesitaban comunicarse con un servidor común (el MCP) en lugar de con cada combinación de LLM y herramienta.

David presentó la idea a Justin Spahr-Summers, y juntos comenzaron a sentar las bases de lo que se convertiría en el **Model Context Protocol (MCP)**. Antes de su lanzamiento público, Anthropic organizó un hackathon interno, y varios equipos adoptaron MCP de inmediato, creando herramientas y nuevas integraciones con Claude Desktop. La facilidad con la que MCP permitía crear y distribuir servidores hizo que el ecosistema floreciera rápidamente.

Con el acceso anticipado de proveedores, MCP se integró en varios IDEs y herramientas de uso diario, lo que impulsó su éxito. Pero esta facilidad también trajo **riesgos y problemas**: algunos servidores se crearon sin autenticación ni autorización, otros filtraron información sensible y algunos simplemente no funcionaron como se esperaba. A pesar de ello, la comunidad creció, estableciendo foros en Reddit y un servidor de Discord para compartir experiencias, mejores prácticas y contribuir a la documentación y los SDK de MCP.

A pesar de la historia de adopción y crecimiento, existía un vacío: muchos tutoriales se centraban en la creación de servidores, pero había pocos recursos sobre cómo construir **clientes MCP**. Este libro surge para llenar ese vacío, ofreciendo a los desarrolladores una guía completa para comprender el protocolo, usar el SDK de Python y participar activamente en el ecosistema MCP.

En pocas palabras, MCP no es solo una API ni un simple estándar: es un **protocolo abierto**, independiente de cualquier SDK o implementación, diseñado para que los agentes descubran herramientas, indicaciones y datos, decidan cuándo y cómo usarlos, y permitan la comunicación bidireccional entre agentes y servidores. Su propósito: facilitar integraciones con LLM, distribuir herramientas y datos, potenciar la colaboración y, al mismo tiempo, aprender de los problemas iniciales de seguridad y estabilidad para mejorar el ecosistema.





**Este notebook está basado en Stratis, K. (2024). *Agentes de IA con MCP*. O’Reilly Media, Inc.**


